In [1]:
import numpy as np


In [26]:
class shallow_nn:
    """ a shallow neural network
    dim
        input = 2
        hidden layer = n
        output = 1
    """    

    def __init__(self, n, param_hidden, param_output, learning_rate, ttl_layer=None) -> None:
        print("enter __init__")
        # fool-proof
        self.training = False
        
        # number of node
        self.num_input = 2
        self.num_hidden = n
        self.num_output = 1
        
        # activation function
        self.param_hidden = param_hidden
        self.param_output = param_output
        
        # hyper param
        self.learning_rate = learning_rate
        
        # init layer
        if ttl_layer == None:
            np.random.seed(0) # for reproducibility
            self.weights_input_hidden = np.random.uniform(size=(self.num_input, self.num_hidden))
            # print(f"\tself.weights_input_hidden.shape = {self.weights_input_hidden.shape}")
            self.bias_hidden = np.random.uniform(size=(1, self.num_hidden))
            self.weights_hidden_output = np.random.uniform(size=(self.num_hidden, self.num_output))
            self.bias_output = np.random.uniform(size=(1, self.num_output))
        else:
            self.weights_input_hidden = ttl_layer[0]
            self.bias_hidden = ttl_layer[1]
            self.weights_hidden_output = ttl_layer[2]
            self.bias_output = ttl_layer[3]

            
        print(f"\tself.weights_input_hidden.shape = {self.weights_input_hidden.shape}")
        print(f"\tself.bias_hidden.shape = {self.bias_hidden.shape}")
        print(f"\tself.weights_hidden_output.shape = {self.weights_hidden_output.shape}")
        print(f"\tself.bias_output.shape = {self.bias_output.shape}")
        print("exit __init__")
  
    def get_layer(self):
        ttl_layer = []
        ttl_layer.append(self.weights_input_hidden)
        ttl_layer.append(self.bias_hidden)
        ttl_layer.append(self.weights_hidden_output)
        ttl_layer.append(self.bias_output)
        return ttl_layer
  
    def act_hidden(self,x):
        """ tanh
        """
        print("enter act_hidden")
        # print(f"\tx = {x}")
        exponient = np.e**(2*self.param_hidden*x)
        # print(f"\texponient = {exponient}")
        y = (exponient-1)/(exponient+1)
        # print(f"\ty = {y}")
        print("exit act_hidden")
        return y
    
    def div_act_hidden(self,x):
        """ tanh
        """
        print("enter div_act_hidden")
        print(f"\tx = {x}")
        exponient = np.e**(self.param_hidden*x)
        print(f"\texponient = {exponient}")
        exponient_minus = np.e**(-self.param_hidden*x)
        print(f"\texponient_minus = {exponient_minus}")
        y = self.param_hidden*4/(exponient+exponient_minus)
        print(f"\ty = {y}")
        print("exit div_act_hidden")
        return y
    
    def act_output(self, x):
        """ linear
        """
        y = self.param_hidden*x
        return y
    
    def div_act_output(self, x):
        """ linear
        """
        y = self.param_hidden
        return np.array([y])
    
    # def loss_function(self, pred, target):
    #     """ compute the error
    #     """
    #     error = target - pred
    #     loss = np.mean(np.square(error))
    #     return loss
    
    def forward(self, X, training=False):
        """
        Var:
            training: bool
                True    backward later
                False   forward only
        """
        print(f"enter forward, training={training}")
        # print(f"\ttype(X) = {type(X)}")
        # print(f"\tX.shape = {X.shape}")
        self.training = training
        if self.training is True:
            print("if self.training is True")
            self.data = X
        else:
            print("if self.training is False")
            self.data = None
        
        print()
        # print(f"\ttype(X) = {type(X)}")
        # print(f"\tX.shape = {X.shape}")
        # print(f"\tself.weights_input_hidden.shape = {self.weights_input_hidden.shape}")
        print(f"X = {X}")
        print(f"self.weights_input_hidden = {self.weights_input_hidden}")
        self.hidden_layer_in = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        print(f"\tself.hidden_layer_in = {self.hidden_layer_in}")
        self.hidden_layer_out = self.act_hidden(self.hidden_layer_in)
        # print(f"\tself.hidden_layer_out = {self.hidden_layer_out}")
        
        # print(f"\tself.hidden_layer_out.shape = {self.hidden_layer_out.shape}")
        # print(f"\tself.weights_hidden_output.shape = {self.weights_hidden_output.shape}")
        self.output_layer_in = np.dot(self.hidden_layer_out, self.weights_hidden_output) + self.bias_output
        # print(f"\tself.output_layer_in.shape = {self.output_layer_in.shape}")
        self.output_layer_out = self.act_output(self.output_layer_in)
        
        
        # print(f"\tself.weights_input_hidden.shape = {self.weights_input_hidden.shape}")
        # print(f"\tself.bias_hidden.shape = {self.bias_hidden.shape}")
        # print(f"\tself.weights_hidden_output.shape = {self.weights_hidden_output.shape}")
        # print(f"\tself.bias_output.shape = {self.bias_output.shape}")
        
        pred = self.output_layer_out
        
        print(f"exit forward, training={training}")
        return pred
    
    def backward(self, target):
        print("enter backward")
        if self.training is False:
            print("\tif self.training is False")
        else:
            print("\tif self.training is True")
            # error = target - self.output_layer_activation
            # loss = 0.5*(np.square(error))
            
            # div_loss_2_error = error
            # div_error_2_output = -1
            
            # compute gradient
            # print(f"\ttarget.shape = {target.shape}")
            # print(f"\tself.output_layer_out = {self.output_layer_out.shape}")
            # print()
            error_output = target - self.output_layer_out
            print("\tafter error_output")
            # print(f"\terror_output.shape = {error_output.shape}")
            # print(f"\tself.output_layer_i.shape = {self.output_layer_in.shape}")
            # print(f"\tself.div_act_output(self.output_layer_in).shape = {self.div_act_output(self.output_layer_in).shape}")
            # print()
            
            back_output = error_output * (-1) * self.div_act_output(self.output_layer_in)
            print("\tafter back_output")
            # print(f"\ttype(back_output) = {type(back_output)}")
            # print(f"\tback_output.shape = {back_output.shape}")
            # print(f"\ttype(self.hidden_layer_out) = {type(self.hidden_layer_out)}")
            # print(f"\tself.hidden_layer_out.shape = {self.hidden_layer_out.shape}")
            # print()
            
            # grad_output = back_output.dot(self.hidden_layer_out.T)
            grad_output = self.hidden_layer_out.T.dot(back_output)
            print("\tafter grad_output")
            # print(f"\ttype(back_output) = {type(back_output)}")
            # print(f"\tback_output.shape = {back_output.shape}")
            # print(f"\ttype(self.weights_hidden_output) = {type(self.weights_hidden_output)}")
            # print(f"\tself.weights_hidden_output.shape = {self.weights_hidden_output.shape}")
            # print(f"\tself.div_act_hidden(self.hidden_layer_in).shape = {self.div_act_hidden(self.hidden_layer_in).shape}")
            # print()

            back_hidden = back_output.dot(self.weights_hidden_output.T) * self.div_act_hidden(self.hidden_layer_in)
            print("\tafter back_hidden")
            print()
            # grad_hidden = back_hidden.dot(self.data.T)
            grad_hidden = self.data.T.dot(back_hidden)
            print("\tafter grad_hidden")
            print()
            
            # update weight and bias
            self.weights_hidden_output -= grad_output * self.learning_rate
            self.bias_output -= np.sum(back_output, axis=0, keepdims=True) * self.learning_rate
            self.weights_input_hidden -= grad_hidden * self.learning_rate
            self.bias_hidden -= np.sum(back_hidden, axis=0, keepdims=True) * self.learning_rate

            # train 過後就關起來
            self.training = False
        print("exit backward")

    def evaluate(self, X, target):
        pred = self.forward(X)
        loss = 0.5*np.mean(np.square(target-pred))
        return loss
    
    def predict(self, X):
        print("enter predict")
        # print(f"\ttype(X) = {type(X)}")
        pred = self.forward(X)
        print("exit predict")
        return pred


##### skip

In [3]:
# normalization
class normalization:
    
    def __init__(self, data) -> None:
        self.data = data
        
        max = np.max(data)
        self.min = np.min(data)
        
        self.range_original = max - self.min
        # range_desired = 1

    
    def normalize(self, data):
        new_data = (data-self.min)/self.range_original
        return new_data
    
    def unnormalize(self, X):
        raw_data = X*self.range_original + self.min
        return raw_data


In [4]:
# construct training data and testing data

np.random.seed(0)
X_size = (20000,2)

X = np.random.randint(1000, 9999, size=X_size)
Y = np.sum(X, axis=1)

rng = np.random.default_rng()
idx_testing = rng.choice(20000, 4000, replace=False, shuffle=False)

# testing data
X_test = X[idx_testing]
Y_test = Y[idx_testing]

# training data
mask = np.ones(Y.shape, dtype=bool)
mask[idx_testing] = False

X_train = X[mask]
Y_train = Y[mask]

display(X)
display(X_train)

array([[3732, 4264],
       [5859, 8891],
       [5373, 6874],
       ...,
       [7168, 9948],
       [5007, 2158],
       [9685, 7721]])

array([[3732, 4264],
       [5859, 8891],
       [5373, 6874],
       ...,
       [7168, 9948],
       [5007, 2158],
       [9685, 7721]])

In [7]:
# preprocess

# normalize
normalization_test = normalization(X_test)
X_test_n = normalization_test.normalize(X_test)
Y_test_n = normalization_test.normalize(Y_test)
Y_test_n = Y_test_n.reshape((-1,1))
display(X_test_n)
display(Y_test_n)


array([[0.76914527, 0.35667445],
       [0.34578193, 0.73468934],
       [0.10681338, 0.74924975],
       ...,
       [0.33011004, 0.05312882],
       [0.0662443 , 0.84661554],
       [0.960987  , 0.49805491]])

array([[1.23696788],
       [1.19161943],
       [0.96721129],
       ...,
       [0.49438702],
       [1.024008  ],
       [1.57019006]])

In [5]:
def get_checkpoint(nn: shallow_nn):
    print("enter seve_checkpoint")
    loss = nn.evaluate(X_test_n, Y_test_n)
    param = nn.get_layer()
    print("exit seve_checkpoint")
    return loss, param 

##### train

In [27]:
# training
nn_1 = shallow_nn(5, 1, 1, 0.01)
ttl_loss = []
ttl_param = []

loss, param = get_checkpoint(nn_1)
ttl_loss.append(loss)
ttl_param.append(param)

for epoch in range(10):
    # Forward pass
    nn_1.forward(X_test_n, training=True)
    
    # BackWard pass
    nn_1.backward(Y_test_n)

    # compute the loss
    loss, param = get_checkpoint(nn_1)
    ttl_loss.append(loss)
    ttl_param.append(param)

    if epoch % 1 == 0:
        print(f'Epoch {epoch} Loss: {loss}')

enter __init__
	self.weights_input_hidden.shape = (2, 5)
	self.bias_hidden.shape = (1, 5)
	self.weights_hidden_output.shape = (5, 1)
	self.bias_output.shape = (1, 1)
exit __init__
enter seve_checkpoint
enter forward, training=False
if self.training is False

X = [[0.76914527 0.35667445]
 [0.34578193 0.73468934]
 [0.10681338 0.74924975]
 ...
 [0.33011004 0.05312882]
 [0.0662443  0.84661554]
 [0.960987   0.49805491]]
self.weights_input_hidden = [[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ]
 [0.64589411 0.43758721 0.891773   0.96366276 0.38344152]]
	self.hidden_layer_in = [[1.44421627 1.23505562 1.3497298  1.68840484 0.53365194]
 [1.45602635 1.09768514 1.43164536 1.82200015 0.49923863]
 [1.33428167 0.93314882 1.30058845 1.70582154 0.40358152]
 ...
 [1.00720948 0.7882346  0.81440165 1.15666631 0.23126056]
 [1.3749048  0.94674027 1.36296308 1.77754391 0.42372832]
 [1.64081841 1.43412506 1.59144425 1.92917926 0.66913774]]
enter act_hidden
exit act_hidden
exit forward, training=Fa

C:\Users\vicky\AppData\Local\Temp\ipykernel_22260\2137258666.py:60: RuntimeWarning: overflow encountered in power
  exponient = np.e**(2*self.param_hidden*x)
C:\Users\vicky\AppData\Local\Temp\ipykernel_22260\2137258666.py:62: RuntimeWarning: invalid value encountered in divide
  y = (exponient-1)/(exponient+1)
C:\Users\vicky\AppData\Local\Temp\ipykernel_22260\2137258666.py:72: RuntimeWarning: overflow encountered in power
  exponient = np.e**(self.param_hidden*x)


In [ ]:
# __init__(self, n, param_hidden, param_output, learning_rate)
nn = shallow_nn(2,1,1,0.01)
# nn.evaluate(X_test_n,Y_test_n)
print(f"type(X_test_n) = {type(X_test_n)}")
print()
pred = nn.predict(X_test_n)
pred

In [143]:
ttl_loss

[1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007690703,
 1.6503305007